In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection

/content/drive/MyDrive/data_science_project/pytorch_tutorial/2_object_detection


In [ ]:
from math import sqrt
from itertools import product

import pandas as pd
import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

#VGG

In [ ]:
# 34층에 걸쳐, vgg모듈을 작성
def make_vgg():
    layers = []
    in_channels = 3 # 색 채널 수
    
    # vgg 모듈에서 사용하는 합성곱 층이나 맥스 풀링의 채널 수 'MC'는 출력에 올림을 수행하는 Max pooling 기본값은 floor(내림)
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256,
          256, 'MC', 512, 512, 512, 'M', 512, 512, 512]
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'MC':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            con2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [con2d, nn.ReLU(inplace=True)] # inplace = True로 두면 입력을 재작성하여 출력으로 바꾼 후 메모리에 유지하지 않음 => 메모리 절약
            in_channels = v
    
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6,
              nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    
    return nn.ModuleList(layers)

In [ ]:
# 동작 확인
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [ ]:
def make_extras():
    layers = []
    in_channels = 1024
    
    # extra 모듈의 합성곱층의 채널수를 설정하는 구성
    
    cfg = [256, 512, 128, 256, 128, 256, 128, 256]
    
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)] # 10 x 10
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)] # 5 x 5
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))] # 3 x 3 
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))] # 1 x 1
    
    return nn.ModuleList(layers)
    
extras_test = make_extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


In [ ]:
# 디폴트 박스의 오프셋을 출력하는 loc_layers,
# 디폴트 박스에 대한 각 클래스의 신뢰도 confidence를 출력하는 conf_layers를 작성
# 각 source 마다 한번씩 연산을 수행한 후 4개, num_class 만큼 잘라서 사용하면 된다.
def make_loc_conf(num_classes=21, bbox_aspect_num=[4, 6, 6, 6, 4, 4]):
    
    loc_layers = []
    conf_layers = []
    
    # VGG의 22층, conv4_3(source1)에 대한 합성곱 층
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[0] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[0] * num_classes, kernel_size=3, padding=1)]
    
    # VGG의 최종층
    loc_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * num_classes, kernel_size=3, padding=1)]
    
    # extra(source3)에 대한 합성곱층
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[2] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[2] * num_classes, kernel_size=3, padding=1)]
    
    # source4
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[3] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[3] * num_classes, kernel_size=3, padding=1)]
    
    # source 5
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[4] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[4] * num_classes, kernel_size=3, padding=1)]
    
    # source 6
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[5] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[5] * num_classes, kernel_size=3, padding=1)]
    
    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)

loc_test, conf_test = make_loc_conf()

print(loc_test)
print(conf_test)
    
    

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


In [ ]:
x = torch.randn(3, 512, 38, 38)
x.size()

torch.Size([3, 512, 38, 38])

In [ ]:
nn.Parameter(torch.Tensor(512)).unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)

tensor([[[[1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34],
          [1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34],
          [1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34],
          ...,
          [1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34],
          [1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34],
          [1.7545e-34, 1.7545e-34, 1.7545e-34,  ..., 1.7545e-34,
           1.7545e-34, 1.7545e-34]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000

In [ ]:
# conC4_3로부터 출력을 scale=20의 L2Norm으로 정규화
# 각 특징량 별로 input 채널에 걸쳐서 L2 정규화
class L2Norm(nn.Module):
    def __init__(self, input_channels=512, scale=20):
        super(L2Norm, self).__init__() # 부모 클래스의 생성자 실행
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale
        self.reset_parameters()
        self.eps = 1e-10
        
    def reset_parameters(self):
        '''
        결합 파라미터의 scale 크기 값으로 초기화 실행
        '''
        init.constant_(self.weight, self.scale) # weight의 값이 모두 scale(=20)
    
    def forward(self, x):
        '''
        38 x 38의 특징량에 대해 512 채널에 걸쳐 제곱합의 루트를 구함.
        각 특징량을 채널에 대해 정규화 한 후 계수를 곱하여 계산
        '''
        
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps # (batch_size, input_channels, 38, 38) -> (batch_size, 1, 38, 38)
        x = torch.div(x, norm)
        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x) # size : (batch, 512, 38, 38)
        out = weights * x
        
        return out 

In [ ]:
# 디폴트 박스를 출력하는 클래스
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()
        
        # 초기 설정
        self.image_size = cfg['input_size'] # 화상 크기 300
        # [38, 19, ...] 각 source의 특징량 맵의 크기
        self.feature_maps = cfg['feature_maps']
        self.num_priors = len(cfg['feature_maps']) # source의 개수 6
        self.steps = cfg['steps'] # [8, 16, ...] DBox의 픽셀 크기
        
        self.min_sizes = cfg['min_sizes']
        # [30, 60, ...] 작은 정사각형의 DBox 픽셀 크기
        self.max_sizes = cfg['max_sizes']
        # [60, 111, ...] 큰 정사각형의 DBox 픽셀 크기
        
        self.aspect_ratios = cfg['aspect_ratios'] # 정사각형의 DBox 화면비 (종횡비)
        
    def make_dbox_list(self):
        '''DBox를 작성한다.'''
        mean = []
        # 'feature_maps' : [38, 19, 10, 5, 3, 1]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                
                # 특징량의 화상 크기(
                # 300 / 'step': [8, 16, 32, 64, 100, 300]
                # step : 각 DBox의 픽셀 크기 => 그림 크기에다 나누면 DBox의 좌표 수(개수)가 나온다
                f_k = self.image_size / self.steps[k]
                
                # DBox 중심 좌표, 0-1로 정규화
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k
                
                # 화면비 1의 작은 DBox [cx, cy, width, height]
                # 'min_sizes' : [30, 60, 111, 152, 213, 264]
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k, s_k]
                
                # 화면비 1의 큰 DBox [cx, cy, width, height]
                # "max_sizes" : [60, 111, 162, 213, 264, 315]
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                # 그 외 화면비의 DBox [cx, cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)] # sqrt 로 각각 나누고 곱함으로써 ar만큼의 비율차이가 나도록 한다.
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        
        # DBox를 텐서로 변환 torch.Size([8732, 4])
        output = torch.Tensor(mean).view(-1, 4)
        
        # DBox가 화상 밖으로 돌출되는 것을 막기 위해,, 크기를 최소 0 최대 1로 제한
        output.clamp_(max=1, min=0)
        
        return output
                

## 동작 확인

In [ ]:
import numpy as np
for i, j in zip([8, 16, 32, 64, 100, 300], [38, 19, 10, 5, 3, 1]):
    print(300 / i)
    print(np.ceil(300 / j))

37.5
8.0
18.75
16.0
9.375
30.0
4.6875
60.0
3.0
100.0
1.0
300.0


In [ ]:

# SSD300 설정
ssd_cfg = {
    'num_classes': 21, # 배경을 포함한 총 클래스 수
    'input_size' : 300, # 화상의 입력 크기
    'bbox_aspect_num' : [4, 6, 6, 6, 4, 4], # 출력할 Box 화면비의 종류
    'feature_maps': [38, 19, 10, 5, 3, 1], # 각 source의 화상의 크기
    'steps': [8, 16, 32, 64, 100, 300] , # DBox의 크기를 정함 step의 크기는 np.ceil(이미지의 크기 / feature map의 크기 )
    'min_sizes': [30, 60, 111, 162, 213, 264], # DBox의 크기(작은 정사각형)
    'max_sizes': [60, 111, 162, 213, 264, 315], # DBox의 크기 (큰 정사각형)
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [3]],
}

dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()

pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
...,...,...,...,...
8727,0.833333,0.833333,0.502046,1.000000
8728,0.500000,0.500000,0.880000,0.880000
8729,0.500000,0.500000,0.961249,0.961249
8730,0.500000,0.500000,1.000000,0.508068


# SSD 클래스 구현

In [ ]:
class SSD(nn.Module):
    """
    SSD 클래스를 구현한다.
    훈련과 추론 시 동작이 다르다.
    """

    
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()
        
        self.phase = phase # train or inference를 지정
        self.num_classes = cfg["num_classes"] # 클래스의 수 21
        
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(
            cfg["num_classes"], cfg["bbox_aspect_num"])
        
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # 추론시는 Detect 클래스를 준비
        if phase == 'inference':
            self.detect = Detect()

# 동작 확인
ssd_test = SSD(phase="train", cfg=ssd_cfg)
print(ssd_test)
            

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [ ]:
# decode 함수
# 오프셋 정보를 이용하여 DBox를 BBox로 변환을 수행
def decode(loc, dbox_list):
    """
    오프셋 정보를 이용하여 DBOx를 BBox로 변환
    
    Parmeters
    ----------
    loc : [8732, 4]
    dbox_list = [8732, 4]
    DBbox 정보
    
    Returns
    --------
    boxes : [xmin, ymin, xmax, ymax]
        BBox 정보
    """
    
    # DBox [cx, cy, width, height]
    # loc [dcx, dcy, dwidth, dheight]
    
    boxes = torch.cat((
        dbox_list[:, :2] + loc[:, :2] * 0.1 * dbox_list[:, 2:],
        dbox_list[:, :2] * torch.exp(loc[:, 2:] * 0.2)), dim=1)
    # boxes의 크기는 torch.Size([8732, 4])가 된다.
    
    # BBox의 좌표정보를 [cx, cy, width, height]에서 [xmin, ymin, xmax, ymax] 변환
    # 순서가 바뀌면 안됨 위에서 
    boxes[:, :2] -= boxes[:, 2:] / 2 # 중심 좌표에서 width / 2, height /2 를 뺌
    boxes[:, 2:] += boxes[:, :2] # 위에서 변환한 x_min, y_min에서 w, h를 각각 더해 x_max, y_max 구현
    
    return boxes

# Non-maximum suppression 구현

In [ ]:
t = torch.randn((2,3))
t_new = t.new()
print(t)
print(t_new)

t_new2 = t.new(3)
print(t_new2)

tensor([[0.6093, 1.0133, 0.2266],
        [1.1165, 1.1868, 0.0426]])
tensor([])
tensor([1.8120e-34, 0.0000e+00, 2.2662e-01])


In [ ]:
t = torch.tensor([1, 5, 10, 3])
t.sort(0)

torch.return_types.sort(values=tensor([ 1,  3,  5, 10]), indices=tensor([0, 3, 1, 2]))

In [ ]:
t

tensor([ 1,  5, 10,  3])

In [ ]:
def nm_suppression(boxes, scores, overlap=0.45, top_k=200):
    """
    Non-Maximum suppression을 실시하는 함수.
    boxes 중에서 겹치는(overlap 이상) BBox 삭제
    
    모든 BBox를 고려하기에는 처리 비용이 커지므로, conf가 높은 상위 k개(top_k)의 BBox(scores)만을 이용하여 수행한다.

    Parameters
    ----------
    boxes : (신뢰도 임계값(0.01)을 넘은 Bbox의 수, 4)
        BBox 정보
    scores: (신뢰도 임계값(0.01)을 넘은 BBox의 수)
        conf 정보

    Returns
    -------
    keep : 리스트
        conf의 내림차순으로 nms를 통과한 index가 저장된
    count : int
        nms를 통과한 BBox 수
    """

    count = 0
    keep = scores.new(scores.size(0)).zero_().long() # new 새로운 크기의 텐서 생성 # new에 숫자를 인자로 넣어주면 device와 type이 일치하는 random을 반환한다.
    # keep : torch.Size([신뢰도 임계값을 넘은 BBox의 수]), 요소는 전부 0

    # 각 BBox의 area 면적을 게산
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2 - x1, y2 - y1)

    # boxes를 복사한다. 나중에 BBox 중복도(IOU)계산시 모형으로 준비
    tmp_x1 = boxes.new() # type과 device만 일치하고 내용이 없는 텐서를 반환한다. 
    tmp_y1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()

    # score를 오름차순으로 나열 (기준 차원)
    v, idx = scores.sort(0)

    # 상위 top_k개 (200개)의 BBox의 index를 꺼낸다.(200개가 존재하지 않을 수도 있다.)
    idx = idx[-top_k:]

    # idx의 요쇼의 수가 0이 아닌 한 루프
    while idx.numel() > 0: # numel : number of element : 텐서의 요소수 반환
        i = idx[-1] # conf의 최대 index를 i로
        # keep의 끝에 conf 최대 index를 저장
        # 이 index의 BBox와 크게 겹치는 BBOx를 삭제

        keep[count] = i
        count += 1

        if idx.size(0) == 1:
            break

        idx = idx[:-1] # 가장 뒤의(가장 높은 conf) index 삭제

        # --------------------
        # keep에 저장한 BBox과 크게 겹치는 BBox를 추출하여 삭제
        # --------------------
        # 하나 감소시킨 idx까지의 BBox를, out으로 지정한 변수로 작성
        torch.index_select(x1, 0, idx, out=tmp_x1) # (input, dim, index, out) index에 상수가 들어가면 거기까지의 인덱스를 의미
        torch.index_select(y1, 0, idx, out=tmp_y1)
        torch.index_select(x2, 0, idx, out=tmp_x2)
        torch.index_select(y2, 0, idx, out=tmp_y2)

        # 모든 BBox에 대해, 현재 BBox=index가 i로 겹치는 값까지로 설정(clamp)
        tmp_x1 = torch.clamp(tmp_x1, min=x1[i])
        tmp_y1 = torch.clamp(tmp_y1, min=y1[i])
        tmp_x2 = torch.clamp(tmp_x2, max=x2[i])
        tmp_y2 = torch.clamp(tmp_y2, max=y2[i])

        # w와 h의 텐서 크기를 index를 하나 줄인 것으로 함
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)

        # clamp한 상태에서 BBox의 폭과 넓이를 구한다.
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1

        # 폭이나 높이가 음수인 것은 0으로 한다.
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)

        # clamp된 상태의 면적을 구한다.
        inter = tmp_w*tmp_h

        # IOU = intersect부분 / (area(a) + area(b) - interset부분)의 계산
        rem_areas = torch.index_select(area, 0, idx) # 원래면적
        union = (rem_areas - inter) + area[i]
        IoU = inter / union
        
        # IoU가 overlap보다 작은 idx만 남긴다. => IoU가 overlap보다 큰 BBox는
        # 해당 BBox중 가장 큰 conf를 갖는 BBox(keep)만 남겨둔다.
        idx = idx[IoU.le(overlap)]

    # while 루프에서 빠져나오면 종료
    return keep, count

# Detect class 구현

In [ ]:
class Detect(Function): # nn.Module 내부에 들어가서 같은 forward 연산을 수행하기 위한 torch.autograd.function 상속

    def __init__(self, conf_thresh=0.01, top_k=200, nms_thresh=0.45):
        self.softmax = nn.Softmax(dim=-1)
        self.conf_thresh = conf_thresh
        self.top_k = top_k
        self.nms_thresh = nms_thresh
    

    def forward(self, loc_data, conf_data, dbox_list):
        """
        순전파 계산 수행

        Parameters
        loc_data: (batch_num, 8732, 4)
        conf_data: (batch_num, 8732, num_classes)
        dbox_list: (8732, 4)
            DBox 정보

        Returns
        output: torch.size(batch_num, 21, 200, 5)
            (batch_num, classes, conf top200, BBox)
        """

        num_batch = loc_data.size(0)
        num_dbox = loc.data.size(1)
        num_classes = conf_data.size(2)

        # softmax를 이용한 conf 정규화
        conf_data = self.softmax(conf_data)

        # 출력 형식 작성
        output = torch.zeros(num_batch, num_classes, self.top_k, 5)

        # (batch, 8732, num_classes) -> (batch, num_classes, 8732)
        conf_preds = conf_data.transpose(2, 1)

        for i in range(num_batch):
            
            # loc과 DBox를 이용하여 BBox 정보 추출
            decoded_boxes = decode(loc_data[i], dbox_list)

            # conf 복사본 작성
            conf_scores = conf_preds[i].clone()

            for cl in range(1, num_classes):
                # 임계값을 넘은 BBox 추출.
                # 임계값을 넘은 conf의 인덱스를 c_maxk로 취득
                c_mask = conf_scores[cl].gt(self.conf_thresh)
                # gt: greater than 임계값을 넘은 것은 1, 아닌 것은 0인 mask 행렬 반환
                # conf_scores:torch.Size([21, 8732]
                # c_mask:torch.Size([8732])

                # scores : torch.Size([임계값을 넘은 BBox 수])
                scores = conf_scores[cl][c_mask]

                # 임계값을 넘은 conf가 없으면 continue
                if scores.nelement() == 0:
                    continue
                
                # c_mask를 decoded boxes에 적용할 수 있도록 크기 변경
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
                # l_mask = torch.Size([8732, 4])

                # l_mask를 decoded_boxes에 적용
                boxes = decoded_boxes[l_mask].view(-1, 4) # decoded_boxes[l_mask]이 1차원이 되기 때문에 view로 차원을 잡아준다.
                # (임계값을 넘은 BBox 수, 4)

                # Non-maximum suppression 실시
                ids, count = nm_suppression(boxes, scores, self.nms_thresh, self.top_k)
                # conf의 내림차순으로 NMS를 통과한 index 저장
                # count: NMS를 통과한 BBox 수

                # output에 NMS를 뺀 결과를 저장
                output[i, cl, :count] = torch.cat((scores[ids[:count]].unsqueeze(1), boxes[ids[:count]]), 1)

        return output


# SSD 클래스 구현

In [ ]:
class SSD(nn.Module):

    def __init__(self, phase, cfg):
        super(SSD, self).__init__()

        self.phase = phase # train or inference 지정
        self.num_classes = cfg["num_classes"] # 클래스 수 = 21

        # SSD 네트워크 작성
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg["num_classes"], cfg["bbox_aspect_num"])

        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()

        if phase == 'inference':
            # 추론 시에는 "Detect" 클래스를 준비한다.
            self.detect = Detect()
        
    def forward(self, x):
        sources = list() # loc와 conf로의 입력 source 1~6을 저장
        loc = list() # loc의 출력을 저장
        conf = list() # conf의 출력을 저장

        # vgg의 conv4_3 까지 저장
        for k in range(23):
            x = self.vgg[k](x)
        
        source1 = self.L2Norm(x)
        sources.append(source1)
        
        # vgg를 끝까지 계산하여, source2를 계산하고, sources에 추가
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)
        
        sources.append(x)

        # extras의 conv와 ReLU를 계산,
        # source 3~6을 sources에 추가
        for k, v in enumerate(self.extras):
            x = F.relu(v(x), inplace=True)
            if k % 2 == 1: # conv -> ReLU -> conv -> ReLU 를 하여 sources에 넣는다.
                sources.append(x)
            
        # source 1~6에 각가 대응하는 합성곱을 1회씩 적용
        # zip으로 for 루프의 여러 리스트 요소를 취득

        for (x, l, c) in zip(sources, self.loc, self.conf):
            loc.append(l(x).permute(0, 2, 3, 1).contiguous()) #
            conf.append(c(x).permute(0, 2, 3, 1).contiguous()) #
            # l(x)와 c(x)으로 합성곱을 실행
            # l(x)와 c(x)의 출력 크기는 [batch_num, 4 * 화면비의 종류 수, featuremap 높이(h), featuremap 폭(w)]
            # source에 따라 화면비의 종류 수가 다르며, 용이하게 처리하기 위해 순서를 교체,
            # permute로 요소 순서 교체
            # [minibatch, featuremap(h),  featuremap(w), 4 * 화면비 종류 수]
            # 이때 메모리의 위치를 연속적으로 유지시켜주기 위해 contiguous() 매서드 사용한다. # 다음으로 view를 사용하기 때문에 변수가 메모리상에 연속적으로 배치되어 있어야 한다.
        
        # loc와 conf의 형태를 변형한다.
        # loc : torch.Size([batch_num, 34928])
        # conf : torch.Size([batch_num, 183372])가 된다.
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)

        # loc과 conf의 형태를 변형
        # loc의 크기는 torch.Size([batch_num, 8732, 4])
        # conf의 크기는 torch.Size([batch_num, 8732, 21])
        loc = loc.view(loc.size(0), -1, 4)
        conf = conf.view(conf.size(0), -1, self.num_classes)

        output = (loc, conf, self.dbox_list)

        if self.phase == "inference": # 추론시
            # "Detect" 클래스의 forward를 수행
            # 변환 값의 크기는 torch.Size([batch_num, 21, 200, 5])
            return self.detect(output[0], output[1], output[2])

        else:
            return output
            # 반환값 (loc, conf, dbox_list)의 튜플

# Scratch Notes

## torch.index_select

In [ ]:
import torch
import numpy as np

In [ ]:
t = torch.rand((10, 4))
print(t)

tensor([[0.7388, 0.5302, 0.4670, 0.0855],
        [0.3849, 0.2438, 0.3271, 0.9116],
        [0.7462, 0.0146, 0.3962, 0.6928],
        [0.6839, 0.8789, 0.7765, 0.4168],
        [0.8539, 0.7782, 0.5335, 0.4903],
        [0.1378, 0.0032, 0.9645, 0.2713],
        [0.5176, 0.0511, 0.3783, 0.8345],
        [0.3981, 0.1544, 0.5320, 0.6904],
        [0.7341, 0.0842, 0.2819, 0.7368],
        [0.9615, 0.5427, 0.7278, 0.9808]])


In [ ]:
idx = t.size(0)
r = torch.index_select(t, 0, torch.tensor(list(range(idx))))
print(r)

tensor([[0.7388, 0.5302, 0.4670, 0.0855],
        [0.3849, 0.2438, 0.3271, 0.9116],
        [0.7462, 0.0146, 0.3962, 0.6928],
        [0.6839, 0.8789, 0.7765, 0.4168],
        [0.8539, 0.7782, 0.5335, 0.4903],
        [0.1378, 0.0032, 0.9645, 0.2713],
        [0.5176, 0.0511, 0.3783, 0.8345],
        [0.3981, 0.1544, 0.5320, 0.6904],
        [0.7341, 0.0842, 0.2819, 0.7368],
        [0.9615, 0.5427, 0.7278, 0.9808]])


In [ ]:
idx = torch.tensor([1, 2, 3])
r = torch.index_select(t, 0, idx)
print(r)

tensor([[0.3849, 0.2438, 0.3271, 0.9116],
        [0.7462, 0.0146, 0.3962, 0.6928],
        [0.6839, 0.8789, 0.7765, 0.4168]])


## Clone

In [ ]:
t = torch.randn((2, 3))
print(t)

tensor([[ 0.7015, -0.9891,  0.0020],
        [ 0.3456, -0.0370, -1.5582]])


In [ ]:
t_c = t[0].clone()
t_c[1] = 3
print(t_c)
print(t)

tensor([7.0145e-01, 3.0000e+00, 1.9556e-03])
tensor([[ 0.7015, -0.9891,  0.0020],
        [ 0.3456, -0.0370, -1.5582]])


In [ ]:
t_c = t.clone()
t_c[1] = 3
print(t_c)
print(t)

tensor([[ 7.0145e-01, -9.8913e-01,  1.9556e-03],
        [ 3.0000e+00,  3.0000e+00,  3.0000e+00]])
tensor([[ 0.7015, -0.9891,  0.0020],
        [ 0.3456, -0.0370, -1.5582]])


## torch.nelement


In [ ]:
t = torch.randn((10, 3))
print(t.nelement())

30


## VGG

In [ ]:
vgg_test = make_vgg()

In [ ]:
vgg_test

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

## torch.contiguous

https://jimmy-ai.tistory.com/122